In [1]:
import os
import pandas as pd
import json
import re
import numpy as np
import time


In [ ]:
from openai import OpenAI
client = OpenAI(api_key="API_TOKEN_OPEN_AI")

In [3]:
excel_file = "textos_resultados.xlsx"
df = pd.read_excel(excel_file)
df.shape



(2000, 6)

In [6]:
df['calificacion_tipo_texto'].value_counts()

calificacion_tipo_texto
1    1195
0     805
Name: count, dtype: int64

In [13]:
excel_file = "textos_resultados.xlsx"
resultados_file = "textos_resultados.xlsx"
progreso_file = "progreso.json"

# Leer el Excel original o el de resultados si ya existe
if os.path.exists(resultados_file):
    df = pd.read_excel(resultados_file)
else:
    df = pd.read_excel(excel_file)
    # df = df[df['Dataset'] == 'test'].reset_index(drop=True)
    df =  df[df['calificacion_tipo_texto'].isna() | (df['calificacion_tipo_texto'] == '')].reset_index(drop=True)
    # df["calificacion_tipo_texto"] = None  # Crear columna si no existe

# Cargar progreso
try:
    with open(progreso_file, "r") as f:
        progreso = json.load(f)
        ultimo_indice = progreso.get("ultimo_indice", 0)
except FileNotFoundError:
    ultimo_indice = 0

# Parámetros
lote_size = 1

if "calificacion_humor" not in df.columns:
    df["calificacion_humor"] = None

pendientes = df[df["calificacion_humor"].isna() & (df["calificacion_tipo_texto"] == 1)].reset_index().rename(columns={"index": "indice_original"})
pendientes.shape

(120, 8)

In [14]:
df.shape

(2000, 7)

In [15]:
for start in range(0, len(pendientes), lote_size):
    end = min(start + lote_size, len(pendientes))
    lote = pendientes.iloc[start:end]


    textos = lote["Texto"].tolist()

    # Simulación de respuesta (reemplaza por llamada real a la API)
    prompt = f"""
      Tu tarea es clasificar el nivel de humor o gracia que puede generar estos textos de humor en una escala del 1 al 5
      
      Debes brindar la respuesta sin texto introductorio asinando una puntuación de 1 a 5 del nivel de gracia que genera el texto, seprado por comas

{chr(10).join(['Texto: ' + i for i in textos])}
      """
    # print(prompt)
    # respuestas = [f"Calificación simulada para texto {i+start}" for i in range(len(textos))]

    # Guardar calificaciones
    # print(respuestas)
    # print(start,end)
    # print(df.loc[start:end-1, "calificacion_tipo_texto"] )
    # print(prompt)
    completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {"role": "system", "content": prompt}
      ]
      )
    response = completion.choices[0].message.content
    print("RESPONSE",response)
    if len(response.split(',')) == len(textos):
      respuestas = response.split(',')
      
      for i, idx_original in enumerate(lote["indice_original"]):
        print(f"{idx_original}")
        df.at[idx_original, "calificacion_humor"] = respuestas[i]
    else:
       print("####ERROR")
       pass
    # Guardar resultados
    df.to_excel(resultados_file, index=False)

    # Guardar progreso
    # with open(progreso_file, "w") as f:
    #     json.dump({"ultimo_indice": end}, f)

    print(f"Lote {start}-{end} procesado.")
    time.sleep(1)  # espera opcional

RESPONSE 4
60


/tmp/ipykernel_10569/2553894954.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx_original, "calificacion_humor"] = respuestas[i]


Lote 0-1 procesado.
RESPONSE 3
61
Lote 1-2 procesado.
RESPONSE 3
62
Lote 2-3 procesado.
RESPONSE 3
63
Lote 3-4 procesado.
RESPONSE 4
64
Lote 4-5 procesado.
RESPONSE 3
65
Lote 5-6 procesado.
RESPONSE 2
66
Lote 6-7 procesado.
RESPONSE 4
67
Lote 7-8 procesado.
RESPONSE 4
68
Lote 8-9 procesado.
RESPONSE 3
69
Lote 9-10 procesado.
RESPONSE 3
70
Lote 10-11 procesado.
RESPONSE 3
71
Lote 11-12 procesado.
RESPONSE 3
72
Lote 12-13 procesado.
RESPONSE 3
73
Lote 13-14 procesado.
RESPONSE 3
74
Lote 14-15 procesado.
RESPONSE 4
75
Lote 15-16 procesado.
RESPONSE 3
76
Lote 16-17 procesado.
RESPONSE 4
77
Lote 17-18 procesado.
RESPONSE 3
78
Lote 18-19 procesado.
RESPONSE 3
79
Lote 19-20 procesado.
RESPONSE 2
200
Lote 20-21 procesado.
RESPONSE 4
201
Lote 21-22 procesado.
RESPONSE 3
202
Lote 22-23 procesado.
RESPONSE 4
203
Lote 23-24 procesado.
RESPONSE 3
204
Lote 24-25 procesado.
RESPONSE 3
205
Lote 25-26 procesado.
RESPONSE 3
206
Lote 26-27 procesado.
RESPONSE 3
207
Lote 27-28 procesado.
RESPONSE 3
208
Lo

In [16]:

pendientes = df[df["calificacion_humor"].isna() & (df["calificacion_tipo_texto"] == 1)].reset_index().rename(columns={"index": "indice_original"})
pendientes.shape

(0, 8)